# Indiana 2008 Presidential Elections: Data Cleaning & Preprocessing

**Goal:** Build a clean, analysis-ready county-level table for Indiana, 2008 by merging the presidential primary and presidential general election results, and then derive summary stats (party totals).

**Output**: A single CSV where each row is a county and columns include:

- Primary per-candidate vote counts (prefixed with `pri_`)
- General per-candidate vote counts (prefixed with `gen_`)
- Party totals: `rep_primary_total`, `dem_primary_total`, `rep_general_total`, `dem_general_total`, `lib_general_total`, `cst_general_total`, `grn_general_total`, `soc_general_total`, `aip_general_total`, `ind_general_total`

**Last Updated**: 2025/10/02

## 0. Library Import

In [1]:
import re
import pandas as pd
import numpy as np
from pathlib import Path

/Users/amourtu1934/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


## 1. Inputs & Parameters

Define raw file paths once here so the entire notebook is easy to rerun on another machine. If a path changes, we only update it here. We keep a single `OUTPUT_PATH` so all exports land in one known place.

In [2]:
# IN 2008 dataset path
PRIMARY_PATH = r"../../data/raw/2008/IN/20080506__in__primary__county.csv"
GENERAL_PATH = r"../../data/raw/2008/IN/20081104__in__general__county.csv"

# Output directory
OUTPUT_PATH  = r"../../data/processed/2008/IN/"

# Analysis parameters
DISPLAY_ROWS = 10   # Number of rows to display in dataframes

## 2. Load & Filter

We load primary and general datasets separately and immediately subset to the rows we truly need:

- Restrict `office` to 'President' to avoid mixing down-ballot contests

- Remove columns that are fully missing or irrelevant post-filter (e.g., a district column that’s empty for county-level rows)

### a. Primary Election Dataset

In [3]:
# Load primary data
primary_df = pd.read_csv(PRIMARY_PATH)
primary_df.head(DISPLAY_ROWS)

,county,office,district,candidate,party,votes
0,Adams,President,NaN,Hillary Clinton,Democratic,3147
1,Adams,President,NaN,Barack Obama,Democratic,2283
2,Adams,President,NaN,Mike Huckabee,Republican,403
3,Adams,President,NaN,John Mccain,Republican,1735
4,Adams,President,NaN,Ron Paul,Republican,292
5,Adams,President,NaN,Mitt Romney,Republican,141
6,Adams,Governor,NaN,Jim Schellinger,Democratic,1717
7,Adams,Governor,NaN,Jill Long Thompson,Democratic,3586
8,Adams,Governor,NaN,Mitch Daniels,Republican,2141
9,Adams,U.S. House,3,Michael A. (Mike) Montagano,Democratic,0


In [4]:
# Different values in 'office' column
primary_df["office"].value_counts()

office
President                      552
U.S. House                     349
State House                    289
Governor                       276
State Senator                  171
Judge Of The Superior Court    139
Judge Of The Circuit Court      46
Judge Of The County Court        3
Name: count, dtype: int64

In [5]:
# Only keep rows where 'office' is 'President'
primary_df = primary_df[primary_df["office"] == "President"]
primary_df.shape

(552, 6)

In [6]:
# Now, drop the "office" column as it's no longer needed
# Also, drop the district column
primary_df = primary_df.drop(columns=["office", "district"]).reset_index(drop=True)
primary_df.head(DISPLAY_ROWS)

,county,candidate,party,votes
0,Adams,Hillary Clinton,Democratic,3147
1,Adams,Barack Obama,Democratic,2283
2,Adams,Mike Huckabee,Republican,403
3,Adams,John Mccain,Republican,1735
4,Adams,Ron Paul,Republican,292
5,Adams,Mitt Romney,Republican,141
6,Allen,Hillary Clinton,Democratic,27535
7,Allen,Barack Obama,Democratic,35103
8,Allen,Mike Huckabee,Republican,2044
9,Allen,John Mccain,Republican,13854


In [7]:
# Unique parties in primary_df
primary_df["party"].value_counts()

party
Republican    368
Democratic    184
Name: count, dtype: int64

In [8]:
# Candidates in primary_df
primary_df["candidate"].value_counts()

candidate
Hillary Clinton    92
Barack Obama       92
Mike Huckabee      92
John Mccain        92
Ron Paul           92
Mitt Romney        92
Name: count, dtype: int64

In [9]:
# Missing values count
primary_df.isnull().sum()

county       0
candidate    0
party        0
votes        0
dtype: int64

In [10]:
# Calculuate the duplicate rows
duplicate_rows = primary_df.duplicated()
duplicate_rows.sum()

0

In [11]:
# Final look at the cleaned primary_df
primary_df.head(DISPLAY_ROWS)

,county,candidate,party,votes
0,Adams,Hillary Clinton,Democratic,3147
1,Adams,Barack Obama,Democratic,2283
2,Adams,Mike Huckabee,Republican,403
3,Adams,John Mccain,Republican,1735
4,Adams,Ron Paul,Republican,292
5,Adams,Mitt Romney,Republican,141
6,Allen,Hillary Clinton,Democratic,27535
7,Allen,Barack Obama,Democratic,35103
8,Allen,Mike Huckabee,Republican,2044
9,Allen,John Mccain,Republican,13854


### b. General Election Dataset

In [12]:
# Load general data
general_df = pd.read_csv(GENERAL_PATH)
general_df.head(DISPLAY_ROWS)

,county,office,district,candidate,party,votes
0,Adams,President,NaN,Barack Obama,Democratic,4928
1,Adams,President,NaN,Bob Barr,Libertarian,170
2,Adams,President,NaN,John Mccain,Republican,8404
3,Adams,President,NaN,Darrell L Castle,Constitution,0
4,Adams,President,NaN,Cynthia A Mckinney,Green,1
5,Adams,President,NaN,Michael L Faith,Americas Independent,0
6,Adams,President,NaN,Chuck Baldwin,Independent,9
7,Adams,President,NaN,Lawson Mitchell Bone,Independent,0
8,Adams,President,NaN,Kevin Mottus,Independent,0
9,Adams,President,NaN,Ralph Nader,Independent,1


In [13]:
# Different values in 'office' column
general_df["office"].value_counts()

office
State Convention Delegate               2354
President                               1196
Governor                                 460
State House                              425
Superintendent Of Public Instruction     276
U.S. House                               260
Attorney General                         184
Judge Of The Superior Court              134
State Senator                            129
Judge Of The Circuit Court                43
Judge Of The County Court                  4
Prosecuting Attorney                       1
Name: count, dtype: int64

In [14]:
# Only keep rows where 'office' is 'President'
general_df = general_df[general_df["office"] == "President"]
general_df.shape

(1196, 6)

In [15]:
# Now, drop the "office" column as it's no longer needed
# Also, drop the district column
general_df = general_df.drop(columns=["office", "district"]).reset_index(drop=True)
general_df.head(DISPLAY_ROWS)

,county,candidate,party,votes
0,Adams,Barack Obama,Democratic,4928
1,Adams,Bob Barr,Libertarian,170
2,Adams,John Mccain,Republican,8404
3,Adams,Darrell L Castle,Constitution,0
4,Adams,Cynthia A Mckinney,Green,1
5,Adams,Michael L Faith,Americas Independent,0
6,Adams,Chuck Baldwin,Independent,9
7,Adams,Lawson Mitchell Bone,Independent,0
8,Adams,Kevin Mottus,Independent,0
9,Adams,Ralph Nader,Independent,1


In [18]:
# Candidates in general_df
general_df["candidate"].value_counts()

candidate
Barack Obama            92
Bob Barr                92
John Mccain             92
Darrell L Castle        92
Cynthia A Mckinney      92
Michael L Faith         92
Chuck Baldwin           92
Lawson Mitchell Bone    92
Kevin Mottus            92
Ralph Nader             92
John Leroy Plemons      92
"Lou" Kujawski          92
Brian Moore             92
Name: count, dtype: int64

In [16]:
# List out all the parties in the general election data
general_df["party"].value_counts()

party
Independent             460
Republican              184
Democratic               92
Libertarian              92
Constitution             92
Green                    92
Americas Independent     92
Socialist                92
Name: count, dtype: int64

In [17]:
# Missing values count
general_df.isnull().sum()

county       0
candidate    0
party        0
votes        0
dtype: int64

In [19]:
# Final look at the cleaned general_df
general_df.head(DISPLAY_ROWS)

,county,candidate,party,votes
0,Adams,Barack Obama,Democratic,4928
1,Adams,Bob Barr,Libertarian,170
2,Adams,John Mccain,Republican,8404
3,Adams,Darrell L Castle,Constitution,0
4,Adams,Cynthia A Mckinney,Green,1
5,Adams,Michael L Faith,Americas Independent,0
6,Adams,Chuck Baldwin,Independent,9
7,Adams,Lawson Mitchell Bone,Independent,0
8,Adams,Kevin Mottus,Independent,0
9,Adams,Ralph Nader,Independent,1


In [20]:
# Shape after preprocessing
general_df.shape

(1196, 4)

## 3. Table Pivoting

We convert tall (one row per county/party/candidate) into wide (one row per county with one column per candidate). This creates the consistent schema with previous group cleaned data.

Helper functions:

- `normalize_party(s)`: in this case, we lower everything so column names are stable with other dataframes
- `candidate_token(name)`: turns “Barack Obama” -> OBAMA, “John McCain” -> MCCAIN, etc. Create a short, readable, unique token for column names
- `pivot_wide(df, prefix, key_col="county")`: Main pivot function
        
    * groups by `county` x `party` × `candidate`, sums `votes`,
    * pivots to columns named like:
        * Primary: `pri_dem_OBAMA`, `pri_rep_MCCAIN`,...
        * General: `gen_dem_OBAMA`, `gen_rep_MCCAIN`,...

    * flattens the MultiIndex into plain column strings,
    * returns one wide row per county

In [21]:
def normalize_party(s: pd.Series) -> pd.Series:
    """
    Normalize party names: Democratic -> dem, Republican -> rep
    """
    return(s.str.strip()
           .str.capitalize()
           .map({
                "Democratic"          : "dem", 
                "Republican"          : "rep",
                "Libertarian"         : "lib",
                "Constitution"        : "cst",
                "Green"               : "grn",
                "Socialist"           : "soc",
                "Americas independent": "aip",
                "Independent"         : "ind",
               })
           .fillna(s.str.strip().str.lower()))      # For defensive purposes only, would not expect other parties

In [22]:
SUFFIXES = {
    "JR","SR","JNR","SNR",
    "II","III","IV","V","VI","VII","VIII","IX","X","XI","XII"
}

def candidate_token(name: str) -> str:
    """
    Turn John McCain -> MCCAIN, Barack Obama -> OBAMA
    Skip suffixes, keep last name/token, capitalize, and remove punctuation
    """
    if pd.isna(name):
        return "UNKNOWN"                # Defensive purposes only, would not expect missing values
    
    # Remove suffixes
    raw = str(name).strip()

    # If a comma exists, treat as 'LAST, FIRST ...'
    if "," in raw:
        last_part = raw.split(",", 1)[0]
        last_part = re.sub(r"[^A-Za-z0-9\s]+", "", last_part).strip().upper()
        tokens = last_part.split()
        return tokens[-1] if tokens else "UNKNOWN"

    # Otherwise: remove punctuation, split, then drop trailing suffixes
    tokens = re.sub(r"[^A-Za-z0-9\s]+", "", raw).strip().upper().split()
    while tokens and tokens[-1] in SUFFIXES:
        tokens.pop()
    return tokens[-1] if tokens else "UNKNOWN"

In [23]:
def pivot_wide(df: pd.DataFrame, prefix: str, key_col: str="county") -> pd.DataFrame:
    """
    Pivot the dataframe to wide format based on party and candidate
    """
    # Normalize party names
    df['party_key'] = normalize_party(df['party'])
    
    # Create candidate tokens
    df['candidate_token'] = df['candidate'].apply(candidate_token)
    
    # Create new column names based on party and candidate token
    df['new_col'] = prefix + '_' + df['party'] + '_' + df['candidate_token']
    
    # Pivot the dataframe
    pivot_df = df.pivot_table(index=key_col, 
                              columns=["party_key", "candidate_token"], 
                              values="votes", 
                              aggfunc='sum', 
                              fill_value=0)
    
    # Flatten multi-level columns
    pivot_df.columns = [f"{prefix}_{p}_{c}" for p, c in pivot_df.columns]
    
    # Reset index to turn key_col back into a column
    pivot_df = pivot_df.reset_index()
    
    return pivot_df

In [24]:
# Primary dataframe pivot
primary_pivot = pivot_wide(primary_df, prefix="pri")
primary_pivot.head(DISPLAY_ROWS)

,county,pri_dem_CLINTON,pri_dem_OBAMA,pri_rep_HUCKABEE,pri_rep_MCCAIN,pri_rep_PAUL,pri_rep_ROMNEY
0,Adams,3147,2283,403,1735,292,141
1,Allen,27535,35103,2044,13854,2824,1418
2,Bartholomew,6903,5989,658,5000,266,380
3,Benton,625,557,177,1124,120,82
4,Blackford,1941,1024,60,406,48,28
5,Boone,4454,4848,500,4747,282,274
6,Brown,2009,1991,144,1076,84,57
7,Carroll,2033,1625,254,1466,117,85
8,Cass,3978,2513,408,2941,243,153
9,Clark,18094,8710,320,2724,128,127


In [25]:
# Primary dataframe shape after pivot
primary_pivot.shape

(92, 7)

In [26]:
# General dataframe pivot
general_pivot = pivot_wide(general_df, prefix="gen")
general_pivot.head(DISPLAY_ROWS)

,county,gen_aip_FAITH,gen_cst_CASTLE,gen_dem_OBAMA,gen_grn_MCKINNEY,gen_ind_BALDWIN,gen_ind_BONE,gen_ind_MOTTUS,gen_ind_NADER,gen_ind_PLEMONS,gen_lib_BARR,gen_rep_KUJAWSKI,gen_rep_MCCAIN,gen_soc_MOORE
0,Adams,0,0,4928,1,9,0,0,1,0,170,0,8404,0
1,Allen,0,0,71263,7,67,0,0,77,0,1114,0,77793,3
2,Bartholomew,0,1,13567,2,18,0,0,29,0,359,0,17067,0
3,Benton,0,0,1563,0,4,0,0,0,0,64,0,2183,0
4,Blackford,0,1,2677,0,2,0,0,3,0,68,0,2690,0
5,Boone,0,1,9752,2,7,0,0,5,0,258,0,16622,0
6,Brown,0,0,3854,4,6,0,0,2,0,129,0,4060,0
7,Carroll,0,1,3736,1,4,0,0,3,0,133,0,4858,0
8,Cass,0,0,7011,0,31,0,0,0,0,265,0,8346,0
9,Clark,11,0,21953,1,9,0,6,13,0,406,0,25326,0


In [27]:
# General dataframe shape after pivot
general_pivot.shape

(92, 14)

## 4. Merge Dataframes

Before merging, we verify that county names match across primary and general:

In [28]:
# Check if county names match between primary_df and general_df
primary_counties = set(primary_df["county"].unique())
general_counties = set(general_df["county"].unique())
common_counties = primary_counties.intersection(general_counties)
print(f"Number of common counties: {len(common_counties)} out of {len(primary_counties)}")

Number of common counties: 92 out of 92


Great. Since we know that all counties name are matched, we don't need to perform further data preprocessing to match the county names. Thus, we can now merge them:

In [29]:
# Merge primary and general dataframes on 'county'
merged_df = primary_pivot.merge(general_pivot, on="county", how="inner").fillna(0)    # There should be no missing values to fill with 0
merged_df.head(DISPLAY_ROWS)

,county,pri_dem_CLINTON,pri_dem_OBAMA,pri_rep_HUCKABEE,pri_rep_MCCAIN,pri_rep_PAUL,pri_rep_ROMNEY,gen_aip_FAITH,gen_cst_CASTLE,gen_dem_OBAMA,gen_grn_MCKINNEY,gen_ind_BALDWIN,gen_ind_BONE,gen_ind_MOTTUS,gen_ind_NADER,gen_ind_PLEMONS,gen_lib_BARR,gen_rep_KUJAWSKI,gen_rep_MCCAIN,gen_soc_MOORE
0,Adams,3147,2283,403,1735,292,141,0,0,4928,1,9,0,0,1,0,170,0,8404,0
1,Allen,27535,35103,2044,13854,2824,1418,0,0,71263,7,67,0,0,77,0,1114,0,77793,3
2,Bartholomew,6903,5989,658,5000,266,380,0,1,13567,2,18,0,0,29,0,359,0,17067,0
3,Benton,625,557,177,1124,120,82,0,0,1563,0,4,0,0,0,0,64,0,2183,0
4,Blackford,1941,1024,60,406,48,28,0,1,2677,0,2,0,0,3,0,68,0,2690,0
5,Boone,4454,4848,500,4747,282,274,0,1,9752,2,7,0,0,5,0,258,0,16622,0
6,Brown,2009,1991,144,1076,84,57,0,0,3854,4,6,0,0,2,0,129,0,4060,0
7,Carroll,2033,1625,254,1466,117,85,0,1,3736,1,4,0,0,3,0,133,0,4858,0
8,Cass,3978,2513,408,2941,243,153,0,0,7011,0,31,0,0,0,0,265,0,8346,0
9,Clark,18094,8710,320,2724,128,127,11,0,21953,1,9,0,6,13,0,406,0,25326,0


In [30]:
# Statistics check on merged dataframe 
merged_df.describe()

,pri_dem_CLINTON,pri_dem_OBAMA,pri_rep_HUCKABEE,pri_rep_MCCAIN,pri_rep_PAUL,pri_rep_ROMNEY,gen_aip_FAITH,gen_cst_CASTLE,gen_dem_OBAMA,gen_grn_MCKINNEY,gen_ind_BALDWIN,gen_ind_BONE,gen_ind_MOTTUS,gen_ind_NADER,gen_ind_PLEMONS,gen_lib_BARR,gen_rep_KUJAWSKI,gen_rep_MCCAIN,gen_soc_MOORE
count,92.000000,92.000000,92.000000,92.000000,92.000000,92.000000,92.000000,92.000000,92.000000,92.000000,92.000000,92.000000,92.000000,92.000000,92.000000,92.000000,92.000000,92.000000,92.000000
mean,7024.804348,6870.358696,447.532609,3481.717391,343.608696,212.836957,0.130435,0.554348,14935.206522,0.945652,11.130435,0.021739,0.097826,9.880435,0.010870,318.010870,0.010870,14626.608696,0.152174
std,9753.806985,16083.991158,480.188343,4961.360021,477.139967,323.263485,1.150417,1.692674,30558.177965,2.288836,15.990531,0.208514,0.696298,17.738542,0.104257,378.622764,0.104257,19528.881718,0.512155
min,625.000000,279.000000,32.000000,199.000000,6.000000,10.000000,0.000000,0.000000,1158.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,47.000000,0.000000,1713.000000,0.000000
25%,2431.750000,1486.750000,168.500000,1187.750000,94.500000,61.500000,0.000000,0.000000,3827.500000,0.000000,0.750000,0.000000,0.000000,1.000000,0.000000,148.500000,0.000000,4520.250000,0.000000
50%,3638.000000,2284.500000,320.000000,2108.000000,160.000000,126.000000,0.000000,0.000000,5835.500000,0.000000,5.500000,0.000000,0.000000,5.000000,0.000000,190.000000,0.000000,8275.000000,0.000000
75%,6753.000000,4822.500000,501.250000,3872.250000,409.500000,230.250000,0.000000,0.000000,11438.250000,1.000000,12.500000,0.000000,0.000000,7.000000,0.000000,324.250000,0.000000,16733.250000,0.000000
max,62476.000000,128360.000000,2985.000000,38042.000000,2824.000000,2207.000000,11.000000,12.000000,241987.000000,15.000000,77.000000,2.000000,6.000000,100.000000,1.000000,2968.000000,1.000000,134313.000000,3.000000


Now, we will add party totals columns: 

- Primary totals:
    * `rep_primary_total` = sum of all `pri_rep_*` columns
    * `dem_primary_total` = sum of all `pri_dem_*` columns

- General totals:
    * `rep_general_total` = sum of all `gen_rep_*` columns
    * `dem_general_total` = sum of all `gen_dem_*` columns
    * `lib_general_total` = sum of all `gen_lib_*` columns
    * `cst_general_total` = sum of all `gen_cst_*` columns
    * `grn_general_total` = sum of all `gen_grn_*` columns
    * `soc_general_total` = sum of all `gen_soc_*` columns
    * `aip_general_total` = sum of all `gen_aip_*` columns
    * `ind_general_total` = sum of all `gen_ind_*` columns

In [31]:
# Add party totals for primary election
rep_primary_cols   = [c for c in merged_df.columns if c.startswith("pri_rep_")]
dem_primary_cols   = [c for c in merged_df.columns if c.startswith("pri_dem_")]

merged_df["rep_primary_total"] = merged_df[rep_primary_cols].sum(axis=1) if rep_primary_cols else 0
merged_df["dem_primary_total"] = merged_df[dem_primary_cols].sum(axis=1) if dem_primary_cols else 0

In [32]:
# Add party totals for general election
rep_general_cols   = [c for c in merged_df.columns if c.startswith("gen_rep_")]
dem_general_cols   = [c for c in merged_df.columns if c.startswith("gen_dem_")]
lib_general_cols   = [c for c in merged_df.columns if c.startswith("gen_lib_")]
cst_general_cols   = [c for c in merged_df.columns if c.startswith("gen_cst_")]
grn_general_cols   = [c for c in merged_df.columns if c.startswith("gen_grn_")]
soc_general_cols   = [c for c in merged_df.columns if c.startswith("gen_soc_")]
aip_general_cols   = [c for c in merged_df.columns if c.startswith("gen_aip_")]
ind_general_cols   = [c for c in merged_df.columns if c.startswith("gen_ind_")]

merged_df["rep_general_total"] = merged_df[rep_general_cols].sum(axis=1) if rep_general_cols else 0
merged_df["dem_general_total"] = merged_df[dem_general_cols].sum(axis=1) if dem_general_cols else 0
merged_df["lib_general_total"] = merged_df[lib_general_cols].sum(axis=1) if lib_general_cols else 0
merged_df["cst_general_total"] = merged_df[cst_general_cols].sum(axis=1) if cst_general_cols else 0
merged_df["grn_general_total"] = merged_df[grn_general_cols].sum(axis=1) if grn_general_cols else 0
merged_df["soc_general_total"] = merged_df[soc_general_cols].sum(axis=1) if soc_general_cols else 0
merged_df["aip_general_total"] = merged_df[aip_general_cols].sum(axis=1) if aip_general_cols else 0
merged_df["ind_general_total"] = merged_df[ind_general_cols].sum(axis=1) if ind_general_cols else 0

In [33]:
# Print out all the column names in the final dataframe
print("Final columns in the cleaned dataframe:")
merged_df.columns

Final columns in the cleaned dataframe:


Index(['county', 'pri_dem_CLINTON', 'pri_dem_OBAMA', 'pri_rep_HUCKABEE',
       'pri_rep_MCCAIN', 'pri_rep_PAUL', 'pri_rep_ROMNEY', 'gen_aip_FAITH',
       'gen_cst_CASTLE', 'gen_dem_OBAMA', 'gen_grn_MCKINNEY',
       'gen_ind_BALDWIN', 'gen_ind_BONE', 'gen_ind_MOTTUS', 'gen_ind_NADER',
       'gen_ind_PLEMONS', 'gen_lib_BARR', 'gen_rep_KUJAWSKI', 'gen_rep_MCCAIN',
       'gen_soc_MOORE', 'rep_primary_total', 'dem_primary_total',
       'rep_general_total', 'dem_general_total', 'lib_general_total',
       'cst_general_total', 'grn_general_total', 'soc_general_total',
       'aip_general_total', 'ind_general_total'],
      dtype='object')

In [34]:
# Preview merged dataframe with totals
merged_df.head(DISPLAY_ROWS)

,county,pri_dem_CLINTON,pri_dem_OBAMA,pri_rep_HUCKABEE,pri_rep_MCCAIN,pri_rep_PAUL,pri_rep_ROMNEY,gen_aip_FAITH,gen_cst_CASTLE,gen_dem_OBAMA,...,rep_primary_total,dem_primary_total,rep_general_total,dem_general_total,lib_general_total,cst_general_total,grn_general_total,soc_general_total,aip_general_total,ind_general_total
0,Adams,3147,2283,403,1735,292,141,0,0,4928,...,2571,5430,8404,4928,170,0,1,0,0,10
1,Allen,27535,35103,2044,13854,2824,1418,0,0,71263,...,20140,62638,77793,71263,1114,0,7,3,0,144
2,Bartholomew,6903,5989,658,5000,266,380,0,1,13567,...,6304,12892,17067,13567,359,1,2,0,0,47
3,Benton,625,557,177,1124,120,82,0,0,1563,...,1503,1182,2183,1563,64,0,0,0,0,4
4,Blackford,1941,1024,60,406,48,28,0,1,2677,...,542,2965,2690,2677,68,1,0,0,0,5
5,Boone,4454,4848,500,4747,282,274,0,1,9752,...,5803,9302,16622,9752,258,1,2,0,0,12
6,Brown,2009,1991,144,1076,84,57,0,0,3854,...,1361,4000,4060,3854,129,0,4,0,0,8
7,Carroll,2033,1625,254,1466,117,85,0,1,3736,...,1922,3658,4858,3736,133,1,1,0,0,7
8,Cass,3978,2513,408,2941,243,153,0,0,7011,...,3745,6491,8346,7011,265,0,0,0,0,31
9,Clark,18094,8710,320,2724,128,127,11,0,21953,...,3299,26804,25326,21953,406,0,1,0,11,28


Now, we save the cleaned dataframe into the processed directory.

In [35]:
# Save the cleaned and merged dataframe to CSV
out_dir = Path(OUTPUT_PATH)
out_dir.mkdir(parents=True, exist_ok=True)
merged_df.to_csv(OUTPUT_PATH + "IN.csv", index=False)